<a href="https://colab.research.google.com/github/anujavenkatachalam04/chvi_vbd_rj/blob/main/notebooks/vbd_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing for Dengue, SR, Malaria data

In [ ]:
# !pip install -r https://raw.githubusercontent.com/anujavenkatachalam04/chvi_vbd_rj/main/requirements.txt

In [2]:
import os
import pandas as pd
import geopandas as gpd
import requests
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
import geopandas as gpd
from shapely.geometry import Point
import uuid
import re
import datetime
from fuzzywuzzy import fuzz, process

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
os.chdir("/content/drive/MyDrive/CHVI")

In [5]:
# Import shapefile
blocks_shp=gpd.read_file("5_Shapefiles/SUBDISTRICT_11/Rajasthan_Blocks.geojson")

In [134]:
# Only retaining reqd cols
blocks_shp=blocks_shp[["geometry", "NAME", "DISTRICT", "C_CODE11"]]

In [8]:
len(blocks_shp)

244

# Dengue 2024

## Dengue - 1st Jan 2024 - 28th Nov 2024
- Out of 34,441 villages in Rajasthan, only 2,878 villages (8.36%) reported any cases of Dengue.

- 9 sub-districts did not report any cases:
('Dungarpur', 'Simalwara')
('Dungarpur', 'Sagwara')
('Dungarpur', 'Aspur')
('Sirohi', 'Reodar')
('Jalor', 'Raniwara')
('Pali', 'Sumerpur')
('Pali', 'Marwar Junction')
('Pali', 'Rohat')
('Sawai Madhopur', '91 (Unknown Sub-district')
('Jodhpur', 'Phalodi')
- All 34 districts reported at least 1 case




In [203]:
# # dengue with patient lat-long
dengue_2024 = gpd.read_file("1_Data/VBD/raw/dengue_patient_locations.gpkg", layer='patients')

In [204]:
dengue_2024.columns

Index(['sample_date', 'geometry'], dtype='object')

In [205]:
# Fix date - should be 2024
dengue_2024["sample_date"] = pd.to_datetime(dengue_2024["sample_date"])

# Replace year < 2024 with 2024
dengue_2024.loc[dengue_2024["sample_date"].dt.year < 2024, "sample_date"] = dengue_2024.loc[dengue_2024["sample_date"].dt.year < 2024, "sample_date"].apply(lambda x: x.replace(year=2024))

In [207]:
# Map dengue cases to blocks
dengue_2024_blocks=gpd.sjoin(dengue_2024, blocks_shp, how="left", predicate="within")

In [208]:
dengue_2024_blocks.isna().sum()

,0
sample_date,0
geometry,0
index_right,0
NAME,0
DISTRICT,0
C_CODE11,0


In [249]:
dengue_2024_blocks=dengue_2024_blocks[["DISTRICT", "NAME","C_CODE11","sample_date"]]

# Dengue 2025: Dec 2024 - Present
- Link to data: [here](https://docs.google.com/spreadsheets/d/1lo7Z4jzo_bLVxBtHD40AzGH514Tv1WfdEGE8wHx-BjM/edit?gid=1495055077#gid=1495055077)
- Balotra & Phalodi subdistricts have not reported any cases in 2025
- 624 cases of which 9 were dropped due to invalid location/date.
- Mapping based on lat-long > patient block name (fuzzy match) > HF block (fuzzy match)



In [209]:
os.listdir("1_Data/VBD/raw/")

['dengue_historical_data_copy.gsheet',
 'KB NVBDCP Malaria Positive Cases Final - Sheet1.csv',
 'malaria_patient_locations.gpkg',
 'dengue_patient_locations.gpkg',
 'Copy of Dengue AXN Taken 2025 Automation Sheet.xlsx']

In [210]:
# download the original data linked above as an excel file and upload to data drive - can also use gsheets API but we need to add it everytime we update the dataset
wb=pd.ExcelFile("1_Data/VBD/raw/Copy of Dengue AXN Taken 2025 Automation Sheet.xlsx")

In [211]:
# clean dates

def string_clean_dates(*, Date) -> datetime.datetime:
    """Nullifies dates with no number, cleans extraneous elements in dates, and converts to datetime format

    Args:
        Date (str or datetime or NaT): date in dataset

    Returns:
        datetime: date in datetime format
    """

    if not re.search(r"[0-9]", str(Date)):
        return pd.NaT

    Date = re.sub(r"\-\-", "-", str(Date))

    formats = ["%d/%m/%y", "%m/%d/%y", "%d/%m/%Y", "%m/%d/%Y"]

    # parse with multiple formats
    for fmt in formats:
        try:
            parsed_date = pd.to_datetime(str(Date), format=fmt)
            return parsed_date
        except ValueError:
            continue

    # If formats above don't work, try parsing with mixed formats - if value error, nullify
    try:
        return pd.to_datetime(Date, format="mixed")
    except ValueError:
        return pd.NaT

In [212]:
# Main processing loop for sheets, cols

skipped = []
main_df = pd.DataFrame()

for sheet in wb.sheet_names:
    if sheet in ["Form Responses 1", "remaining data", "Pending Dengue Cases in Linelis"]:
        continue

    try:
        data = pd.read_excel(wb, sheet, skiprows=2)
    except Exception as e:
        print(f"Sheet: {sheet}. Error reading: {e}. Skipped.")
        skipped.append(sheet)
        continue

    if "S.No" not in data.columns:
        print(f"Sheet: {sheet}. Columns not aligned. Skipped.")
        skipped.append(sheet)
        continue

    if len(data) < 1:
        print(f"Sheet: {sheet}. No data found. Skipped.")
        skipped.append(sheet)
        continue

    data = data.rename(columns={
        "District (Patient)": "District(Patient)",
        "Sub District (Patient)": "Sub District(Patient)"
    })

    data = data.dropna(subset=["Patient Name"])

    try:
        data_filtered = data.dropna(
            subset=["Sub District(Patient)", "Lat-Long", "H F Block", "Sample Collected Date", "Test Performed Date"],
            how="all"
        )
        if len(data) - len(data_filtered) > 0:
            print(f"{sheet}: {len(data) - len(data_filtered)} rows dropped")
        data_filtered = data_filtered[[
            "District(Patient)", "Sub District(Patient)", "Lat-Long",
            "Sample Collected Date", "Test Performed Date", "H F Block", 'Action Taken Date', 'Urban/\n Rural', 'Total House Visit','Total House Positive', 'Total container check',
            'Total container Positive', 'No of Total Source Reducted Container','BTI', 'House', 'Room','No of fooging Activity Sites', 'Post']]
    except KeyError as e:
        print(f"Sheet: {sheet}. Missing columns: {e}. Skipped.")
        skipped.append(sheet)
        continue

    if len(data_filtered) > 0:

        main_df = pd.concat([main_df, data_filtered], ignore_index=True)
    else:
        print(f"Sheet: {sheet}. No valid rows after filtering. Skipped.")
        skipped.append(sheet)


/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell AD14 is marked as a date but the serial value 6715097.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


Sheet: SheetNames. Columns not aligned. Skipped.
Sheet: Sheet63. Columns not aligned. Skipped.
Sheet: Sheet64. Columns not aligned. Skipped.


In [213]:
# Logical date checks

def try_swap_day_month(date):
    """Swap day and month if valid, else return None."""
    if pd.isna(date):
        return None
    try:
        return datetime.datetime(date.year, date.day, date.month)
    except:
        return None

def fix_sample_test_date(sample_date, test_date):
    """
    Fix sample/test date by checking if it falls in range (30-Nov-2024, 30-Jun-2025).
    Try original sample date, then swapped. Then do the same for test date.
    """
    start_date = datetime.datetime(2024, 11, 30)
    end_date = datetime.datetime(2025, 8, 28) # CHANGE TO TODAY

    def in_range(d):
        return d and start_date < d < end_date

    # Try sample date first
    if pd.notna(sample_date):
        if in_range(sample_date):
            return sample_date
        swapped_sample = try_swap_day_month(sample_date)
        if in_range(swapped_sample):
            return swapped_sample

    # Try test date next
    if pd.notna(test_date):
        if in_range(test_date):
            return test_date
        swapped_test = try_swap_day_month(test_date)
        if in_range(swapped_test):
            return swapped_test

    return None

In [214]:
len(main_df)

3823

In [215]:
main_df.columns

Index(['District(Patient)', 'Sub District(Patient)', 'Lat-Long',
       'Sample Collected Date', 'Test Performed Date', 'H F Block',
       'Action Taken Date', 'Urban/\n Rural', 'Total House Visit',
       'Total House Positive', 'Total container check',
       'Total container Positive', 'No of Total Source Reducted Container',
       'BTI', 'House', 'Room', 'No of fooging Activity Sites', 'Post'],
      dtype='object')

In [216]:
# apply clean dates functions
main_df['Sample Collected Date']=main_df['Sample Collected Date'].apply(lambda x: string_clean_dates(Date=x))

In [217]:
main_df['Test Performed Date']=main_df['Test Performed Date'].apply(lambda x: string_clean_dates(Date=x))

In [218]:
main_df["Action Taken Date"]=main_df["Action Taken Date"].apply(lambda x: string_clean_dates(Date=x))

In [219]:
# apply logical date fix functions
main_df["sample_date"] = main_df.apply(lambda row: fix_sample_test_date(row["Sample Collected Date"], row["Test Performed Date"]), axis=1)

In [220]:
main_df=main_df.dropna(subset=["sample_date"])

In [221]:
# check if any dates are postdated based on today's date
main_df[main_df["sample_date"]>=datetime.datetime.today()]

,District(Patient),Sub District(Patient),Lat-Long,Sample Collected Date,Test Performed Date,H F Block,Action Taken Date,Urban/\n Rural,Total House Visit,Total House Positive,Total container check,Total container Positive,No of Total Source Reducted Container,BTI,House,Room,No of fooging Activity Sites,Post,sample_date


In [222]:
# check if any cases are prior to the expected min date
main_df[main_df["sample_date"]<pd.to_datetime("2024-11-30")]

,District(Patient),Sub District(Patient),Lat-Long,Sample Collected Date,Test Performed Date,H F Block,Action Taken Date,Urban/\n Rural,Total House Visit,Total House Positive,Total container check,Total container Positive,No of Total Source Reducted Container,BTI,House,Room,No of fooging Activity Sites,Post,sample_date


In [224]:
# Convert string lat-long entries to shapely Point objects
def latlong_to_point(value):
    if pd.isna(value):
        return None
    try:
        parts = value.split(",")
        if len(parts) != 2:
            return None  # malformed entry
        lat, lon = float(parts[0].strip()), float(parts[1].strip())
        return Point(lon, lat)  # Note: Point(x=lon, y=lat)
    except Exception:
        return None

# Apply conversion
main_df["geometry"] = main_df["Lat-Long"].apply(latlong_to_point)

In [225]:
# Convert to GeoDataFrame and set CRS
main_gdf = gpd.GeoDataFrame(main_df, geometry="geometry", crs="EPSG:4326")

EPSG:4326


,District(Patient),Sub District(Patient),Lat-Long,Sample Collected Date,Test Performed Date,H F Block,Action Taken Date,Urban/\n Rural,Total House Visit,Total House Positive,...,Total container Positive,No of Total Source Reducted Container,BTI,House,Room,No of fooging Activity Sites,Post,sample_date,Patient_ID_created,geometry
0,Ajmer,Ajmer,"26.491502970721488, 74.5508086798137",2025-01-01,2025-01-01,SRINAGAR,2025-02-01,R,100.0,0.0,...,0.0,1.0,0.0,100.0,297.0,0.0,ANM,2025-01-01,30375dd8-1e33-448a-8709-ed23953c6aec,POINT (74.55081 26.4915)
1,Ajmer,Ajmer,"26.464100857307713, 74.62866765287218",2025-02-01,2025-02-01,SRINAGAR,2025-03-01,U,100.0,1.0,...,1.0,2.0,0.0,100.0,321.0,0.0,ANM,2025-02-01,67db7f62-665e-4d86-8606-41e93d9831aa,POINT (74.62867 26.4641)
2,Ajmer,Ajmer,"26.36520998898628, 74.65138241063207",2025-05-01,2025-05-01,AJMER,2025-04-01,R,100.0,0.0,...,0.0,4.0,0.0,100.0,223.0,0.0,ANM,2025-05-01,a625ed0b-7160-4dfc-aee4-108a7a3b5ce4,POINT (74.65138 26.36521)
3,Ajmer,Ajmer,"26.42888668255127, 74.63631415329868",2025-11-01,2025-11-01,SRINAGAR,2025-12-01,U,100.0,1.0,...,1.0,2.0,0.0,100.0,321.0,0.0,ANM,2025-01-11,3529f771-ddbf-4067-bf75-97f7c470fa34,POINT (74.63631 26.42889)
4,Ajmer,Ajmer,"26.455207385803075, 74.62588365940346",2025-01-29,2025-01-29,AJMER,2025-01-30,U,100.0,1.0,...,1.0,2.0,0.0,100.0,321.0,0.0,ANM,2025-01-29,c234e817-caf8-4edd-93b7-65276f8adbb9,POINT (74.62588 26.45521)


In [226]:

print(main_gdf.crs)
main_gdf.sample()

EPSG:4326


,District(Patient),Sub District(Patient),Lat-Long,Sample Collected Date,Test Performed Date,H F Block,Action Taken Date,Urban/\n Rural,Total House Visit,Total House Positive,...,Total container Positive,No of Total Source Reducted Container,BTI,House,Room,No of fooging Activity Sites,Post,sample_date,Patient_ID_created,geometry
1927,Jaipur,NaN,"26.92129946350465, 75.81660684343346",2025-04-09,2025-04-09,AMBER,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-09,6523a63d-9e1c-4036-92d2-487cd9233a19,POINT (75.81661 26.9213)


In [227]:
merged_gdf=main_gdf.sjoin(blocks_shp, how="left", predicate="within")

In [228]:
merged_gdf.columns

Index(['District(Patient)', 'Sub District(Patient)', 'Lat-Long',
       'Sample Collected Date', 'Test Performed Date', 'H F Block',
       'Action Taken Date', 'Urban/\n Rural', 'Total House Visit',
       'Total House Positive', 'Total container check',
       'Total container Positive', 'No of Total Source Reducted Container',
       'BTI', 'House', 'Room', 'No of fooging Activity Sites', 'Post',
       'sample_date', 'Patient_ID_created', 'geometry', 'index_right', 'NAME',
       'DISTRICT', 'C_CODE11'],
      dtype='object')

In [229]:
# missing subdistrict
merged_gdf["NAME"].isna().sum()

np.int64(300)

In [230]:
mapped_gdf=merged_gdf.dropna(subset=["DISTRICT", "NAME"], how="any")

In [231]:
missing_gdf=merged_gdf.loc[merged_gdf["NAME"].isna()]

In [232]:
missing_gdf=missing_gdf[['District(Patient)', 'Sub District(Patient)', 'Lat-Long','H F Block',
       'sample_date', 'Patient_ID_created', 'geometry', 'Action Taken Date', 'Urban/\n Rural', 'Total House Visit','Total House Positive', 'Total container check',
            'Total container Positive', 'No of Total Source Reducted Container','BTI', 'House', 'Room','No of fooging Activity Sites', 'Post']]

In [233]:
import pandas as pd

# Clean: Strip and lowercase for consistent matching
missing_gdf["District(Patient)"] = missing_gdf["District(Patient)"].str.strip()
missing_gdf["Sub District(Patient)"] = missing_gdf["Sub District(Patient)"].str.strip()
missing_gdf["H F Block"] = missing_gdf["H F Block"].str.strip()

# Replace NaN with empty string for safer checks (optional)
missing_gdf["Sub District(Patient)"] = missing_gdf["Sub District(Patient)"].fillna("")
missing_gdf["District(Patient)"] = missing_gdf["District(Patient)"].fillna("")
missing_gdf["H F Block"] = missing_gdf["H F Block"].fillna("")

# Step 1: Fill missing or invalid subdistricts with HF Block
invalid_subdistricts = ["UT", "NOT FOUND", "Not Detected", ""]
mask_invalid = missing_gdf["Sub District(Patient)"].isin(invalid_subdistricts)
missing_gdf.loc[mask_invalid, "Sub District(Patient)"] = missing_gdf.loc[mask_invalid, "H F Block"]

# Step 2: Manual corrections for district names
district_corrections = {
    "Dholpur": "Dhaulpur",
    "Salumber": "Udaipur",
    "Alwar( Tonk)": "Alwar",
    "DIDWANA-KUCHAMAN": "Nagaur",
    "BANSUR KOTPUTALI":"Alwar",
    "KOTPUTLI-BEHAROR":"Alwar"
}
missing_gdf["District(Patient)"] = missing_gdf["District(Patient)"].replace(district_corrections)

# Step 3: Manual corrections for sub-district names
subdistrict_corrections = {
    "Aandhi": "Amber",
    "ULB UDAIPUR": "Girwa",
    "bhinder": "Girwa",
    "SRINAGAR": "Nasirabad",
    "BANSUR KOTPUTALI": "Bansur"

}
missing_gdf["Sub District(Patient)"] = missing_gdf["Sub District(Patient)"].replace(subdistrict_corrections)


In [234]:
missing_gdf["Sub District(Patient)"]=missing_gdf["Sub District(Patient)"].fillna(missing_gdf["H F Block"])

In [235]:
def fuzzy_match_block(missing_gdf, blocks_shp, district_col='District(Patient)', subdistrict_col='Sub District(Patient)', score_cutoff=95):
    matched_rows = []

    for idx, row in missing_gdf.iterrows():
        patient_district = row[district_col]
        patient_subdistrict = row[subdistrict_col]

        # --- Step 1: Fuzzy match district ---
        district_choices = blocks_shp['DISTRICT'].unique()
        matched_district, district_score = process.extractOne(
            patient_district, district_choices, scorer=fuzz.token_sort_ratio
        )

        if district_score < score_cutoff:
            matched_rows.append({
                **row,
                'matched_dtname': None,
                'matched_sdtname': None,
            })
            continue

        # --- Step 2: Filter blocks_shp to matched district ---
        district_blocks = blocks_shp[blocks_shp['DISTRICT'] == matched_district]
        subdistrict_choices = district_blocks['NAME'].unique()

        # --- Step 3: Fuzzy match subdistrict ---
        matched_subdistrict, subdistrict_score = process.extractOne(
            patient_subdistrict, subdistrict_choices, scorer=fuzz.token_sort_ratio
        )

        if subdistrict_score >= score_cutoff:
            matched_rows.append({
                **row,
                'matched_dtname': matched_district,
                'matched_sdtname': matched_subdistrict,
            })
        else:
            matched_rows.append({
                **row,
                'matched_dtname': matched_district,
                'matched_sdtname': None
            })

    return pd.DataFrame(matched_rows)


In [236]:
result_df = fuzzy_match_block(missing_gdf, blocks_shp)

In [237]:
result_df=result_df[~result_df["matched_sdtname"].isna()]

In [238]:
result_df=result_df.rename(columns={
    "matched_dtname":"DISTRICT",
    "matched_sdtname":"NAME"
})

In [239]:
result_df=result_df.merge(blocks_shp.drop(columns="geometry"), on=["DISTRICT", "NAME"], how="left")

In [240]:
dengue_2025=pd.concat([mapped_gdf,result_df])

In [241]:
# NAME, DISTRICT, C_CODE11 must not be null
dengue_2025.isna().sum()

,0
District(Patient),0
Sub District(Patient),186
Lat-Long,220
Sample Collected Date,231
Test Performed Date,231
H F Block,404
Action Taken Date,660
Urban/\n Rural,534
Total House Visit,487
Total House Positive,504


In [250]:
dengue_2025_cases=dengue_2025[["DISTRICT", "NAME", "C_CODE11", "sample_date"]]

In [251]:
dengue_2025_cases.isna().sum()

,0
DISTRICT,0
NAME,0
C_CODE11,0
sample_date,0


In [244]:
# check dates for 2024 & 2025
dengue_2025_cases["sample_date"].min()

Timestamp('2024-12-10 00:00:00')

In [247]:
dengue_2024_blocks["sample_date"].max()

Timestamp('2024-11-28 00:00:00')

In [248]:
dengue_2024_blocks.columns

Index(['sample_date', 'geometry', 'index_right', 'NAME', 'DISTRICT',
       'C_CODE11'],
      dtype='object')

In [283]:
# append files
dengue=pd.concat([dengue_2024_blocks, dengue_2025_cases])

In [284]:
dengue.isna().sum()

,0
DISTRICT,0
NAME,0
C_CODE11,0
sample_date,0


In [ ]:
dengue_block=dengue.groupby(by=["DISTRICT", "NAME", "C_CODE11", "sample_date"]).size().reset_index().rename(columns={0:"dengue_cases"})

In [286]:
# export clean file (sample date, block)
dengue_block.to_csv("1_Data/VBD/clean/dengue_cases_2024_2025.csv", index=False)

In [291]:
# export monthly cases, block
dengue["Year_Month"]=dengue["sample_date"].dt.to_period("M")

In [301]:
dengue_month=dengue.groupby(by=["DISTRICT", "NAME", "C_CODE11","Year_Month"]).size().reset_index().rename(columns={0:"dengue_cases"})

In [302]:
dengue_month.to_csv("1_Data/VBD/clean/dengue_monthwise_2024_2025.csv", index=False)

In [303]:
# weekly cases
dengue["Year"] = dengue["sample_date"].dt.isocalendar().year
dengue["Week"] = dengue["sample_date"].dt.isocalendar().week
dengue["Year_Week"] = dengue["Year"].astype(str) + "-W" + dengue["Week"].astype(str).str.zfill(2)

In [304]:
dengue

,DISTRICT,NAME,C_CODE11,sample_date,Year_Month,Year,Week,Year_Week
0,Jaipur,Jaipur,0811000546000000,2024-10-14,2024-10,2024,42,2024-W42
1,Kota,Ladpura,0812700667000000,2024-11-28,2024-11,2024,48,2024-W48
2,Jhunjhunun,Jhunjhunun,0810300487000000,2024-10-13,2024-10,2024,41,2024-W41
3,Ajmer,Ajmer,0811900607000000,2024-10-13,2024-10,2024,41,2024-W41
4,Tonk,Deoli,0812000620000000,2024-10-12,2024-10,2024,41,2024-W41
...,...,...,...,...,...,...,...,...
226,Tonk,Tonk,0812000618000000,2025-08-10,2025-08,2025,32,2025-W32
227,Udaipur,Girwa,0813000689000000,2025-03-01,2025-03,2025,9,2025-W09
228,Udaipur,Girwa,0813000689000000,2025-01-14,2025-01,2025,3,2025-W03
229,Udaipur,Girwa,0813000689000000,2025-07-29,2025-07,2025,31,2025-W31


In [305]:
weekly_df = (
    dengue.groupby(["DISTRICT", "NAME", "C_CODE11", "Year", "Week", "Year_Week"], as_index=False)
      .size()
      .sort_values(["DISTRICT", "NAME", "Year", "Week"])
)

In [307]:
weekly_df.rename(columns={"size":"dengue_cases"}, inplace=True)

In [309]:
weekly_df.to_csv("1_Data/VBD/clean/dengue_weekwise_2024_2025.csv", index=False)

# Dengue Source Reduction

- dependencies from above

In [310]:
sr=dengue_2025[["DISTRICT", "NAME",  "C_CODE11", "sample_date", 'Action Taken Date',  'Total House Visit','Total House Positive', 'Total container check',
            'Total container Positive', 'No of Total Source Reducted Container','BTI', 'House', 'Room','No of fooging Activity Sites', 'Post']]

In [311]:
for col in ['Total House Visit','Total House Positive', 'Total container check',
            'Total container Positive', 'No of Total Source Reducted Container','BTI', 'House', 'Room','No of fooging Activity Sites', 'Post']:
            print(col, sr[col].unique())

Total House Visit [100.0 nan 97.0 92.0 39 35 'Patient Not Trace' 50
 'Jatiyana, Umrain (Haldighati Army Kent Jaipur)' 41 52 22 30 40 59 65 32
 60 47 20 'UT' 48 53 51 44 57 'Ramgarh Mod Jaipur' 23 54 72 42 17.0 15.0
 13.0 45.0 26.0 10.0 37.0 16.0 38.0 12.0 8.0 19.0 18.0 25.0 1.0 49.0 2.0
 55 205 70.0 28.0 'unkown' 80 61.0 62.0 68.0 67.0 56.0 79.0 84.0 76.0 81.0
 87.0 77.0 73.0 63.0 75 71 85 74 82 90 89 95 770
 'Patient doing MBBS at Bikaner Medical College and residing last 2 years at Bikaner'
 98 88 78 107.0 109.0 133.0 108.0 625.0 411.0 369.0 415.0 46 34 64.0 69.0
 58 5 '-' 94 43 0 105 36.0 27.0 21.0 31.0 33.0 29.0
 'Patient Address is not completed and mobile no is wrong'
 'Patient Address is not completed and mobile no is not mention'
 'He is not belong To Karauli Dist. He lives in Gangapur City Dist Swai Madhopur. So please Remove this Case in Karauli Dist.'
 'Patient Live in Dausa Dist Last 03 year ( Bhandarej Mod Dausa)'
 'Patinet Live in H. No 70/ 227  Sector 07 Pratap Nagar Nea

In [312]:
import re
import numpy as np
import pandas as pd

def extract_number(val):
    """
    Extract the first number from a string/int/float.
    If no number is found, return np.nan
    """
    if pd.isna(val):
        return np.nan
    # if it's already numeric, return as float
    if isinstance(val, (int, float)):
        return float(val)
    # convert to string and find numbers
    val_str = str(val)
    match = re.search(r"\d+\.?\d*", val_str)  # matches int/float
    if match:
        return float(match.group())
    return np.nan


def clean_numeric_columns(df, columns):
    """
    Apply extract_number to selected columns of a dataframe.
    """
    df = df.copy()
    for col in columns:
        df[col] = df[col].apply(extract_number)
    return df


In [313]:
cols_to_clean = [
    "Total House Visit",
    "Total House Positive",
    "Total container check",
    "Total container Positive",
    "No of Total Source Reducted Container",
    "BTI",
    "House",
    "Room",
    "No of fooging Activity Sites"
]

sr_cleaned = clean_numeric_columns(sr, cols_to_clean)


In [314]:
sr_cleaned["Post"].unique()

array(['ANM', 'CHO', 'ASHA', nan, 'LHV', 0, 'DBC', 'NUSINGH OFFICER',
       'Jatiyana, Umrain (Haldighati Army Kent Jaipur)', 'Anm', 'UT',
       'Ramgarh Mod Jaipur', 'Dr.', 'mi', 'Asha', 'ANM & Asha',
       'ANM CHO,DBC ASHA', 'unkown', 'MPHW', 'ANM and ASHA', 'MO', '-',
       'MOIC', 'anm', 'ANM &asha', 'ANM        ASHA',
       'nersing ooficer                                    ASHA              ASHA',
       'ANM     ASHA', ' MO     ANM          ', 'ANM      CHO', 'MPW',
       'asha', 'asha9875130511', 'ASHA ', 'A.N.M', 'amm', 'ANM ', 'Asha ',
       'ANM\nASHA', 'ANM ASHA', 'ANM,ASHA', 'asha ', 'cho', 'Anm\nAsha',
       'anm ', 'aasha', 'anm,ASHA', 'Asha, Anm', 'CHO/ANM', 'NO', 'PHM',
       'Nursing Officer', 'MO/IC', 7627080488, 9001385977, 9001210454,
       '9887886936 9875706236', 'GNM', 'ANM, ASHA', 'CHO, ANM', 'M.I.',
       'SLT', 'No', 'ANM/ASHA', 'ANM \n Asha', 'ASHA &ANM', 'ASHA DBC',
       'ASHA, CHO  &ANM', 'ASHA, DBC &ANM', 'ANM/Asha', 'Not Traceble',
       

In [315]:
import re
import numpy as np
import pandas as pd

def standardize_worker(val):
    """
    Standardize worker role values:
    - Convert to uppercase
    - Map known variations to standard labels
    - Keep unknown tokens (uppercased)
    - Return NaN for junk/missing values
    """
    if pd.isna(val):
        return np.nan

    s = str(val).strip().upper()

    # handle invalid/missing cases
    if s in ["-", "UT", "UNKOWN", "NOT TRACEBLE", "UNTRACEBLE", "NOT DECTED", "NOT FOUND", ""]:
        return np.nan

    # remove digits but keep letters
    s = re.sub(r"\d+", "", s)

    # replace separators with space
    s = s.replace("\n", " ").replace(",", " ").replace("/", " ").replace("&", " ")

    # tokenize
    tokens = s.split()

    # mapping for roles
    mapping = {
        "ANM": "ANM",
        "ASHA": "ASHA",
        "CHO": "CHO",
        "MO": "MO",
        "MOIC": "MOIC",
        "LHV": "LHV",
        "DBC": "DBC",
        "MPW": "MPW",
        "MPHW": "MPHW",
        "N.O": "NURSING OFFICER",
        "NO": "NURSING OFFICER",
        "NURSING": "NURSING OFFICER",
        "MI": "MI",
        "M.I.": "MI"
    }

    standardized = []
    for tok in tokens:
        if tok in mapping:
            standardized.append(mapping[tok])
        else:
            standardized.append(tok)  # keep unknowns as uppercase

    if not standardized:
        return np.nan

    # remove duplicates, sort for consistency
    return ", ".join(sorted(set(standardized)))


def clean_worker_column(df, column):
    """
    Apply standardization to a worker role column.
    """
    df = df.copy()
    df[column] = df[column].apply(standardize_worker)
    return df


In [316]:
sr_cleaned_2 = clean_worker_column(sr_cleaned, "Post")

In [317]:
sr_cleaned_2["Action_Days"]=sr_cleaned_2["Action Taken Date"]-sr_cleaned_2["sample_date"]

In [318]:
sr_cleaned_2.to_csv("1_Data/VBD/clean/source_reduction_2025.csv", index=False)

# District Summary - Dengue
- dependencies from above

In [322]:
# Load all sheets
xls = pd.ExcelFile(f"1_Data/VBD/raw/dengue_historical_data_copy.xlsx")
sheet_names = xls.sheet_names

# # Read and append all sheets

main_df=pd.DataFrame()

for sheet in sheet_names:
  df = pd.read_excel(f"1_Data/VBD/raw/dengue_historical_data_copy.xlsx", sheet_name=sheet, usecols=['District', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug',
       'Sep', 'Oct', 'Nov', 'Dec'])
  df=df.melt(id_vars="District", var_name="Month Name", value_name="Dengue Cases")
  df=df.dropna(subset=["Month Name", "District"], how="any")
  df["Year"]=sheet
  df['month_num'] = pd.to_datetime(df['Month Name'], format='%b').dt.month
  df['Year_Month'] = pd.to_datetime(dict(year=df['Year'], month=df['month_num'], day=1)).dt.to_period('M')
  df.drop(columns=["Month Name", "Year", "month_num"], inplace=True)
  df["District"]=df["District"].str.capitalize().str.strip()
  main_df=pd.concat([main_df, df])


In [323]:
# change districts to match shape files

# Your mapping dictionary
block_to_district_map = {
    "Chittorgarh": "Chittaurgarh",
    "Dholpur": "Dhaulpur",
    "Jalore": "Jalor",
    "Jhunjhunu": "Jhunjhunun",
    "S. madhopur": "Sawai Madhopur",
    "S.ganganagar": "Ganganagar",
    "Anupgarh": "Ganganagar",
    "Balotra": "Barmer",
    "Beawar": "Ajmer",
    "Deeg": "Bharatpur",
    "Didwana": "Nagaur",
    "Dudu": "Jaipur",
    "Gangapur city": "Sawai Madhopur",
    "Jaipur rural": "Jaipur",
    "Jodhpur rural": "Jodhpur",
    "Kekri": "Ajmer",
    "Khairthal": "Alwar",
    "Kotputli": "Jaipur",
    "Neem ka thana": "Sikar",
    "Phalodi": "Jodhpur",
    "Salumber": "Udaipur",
    "Sanchore": "Jalor",
    "Shahpura": "Bhilwara"
}

# Replace values in 'dtname' where key exists, otherwise keep the original
main_df['New_District'] = main_df['District'].replace(block_to_district_map)


In [324]:
hist_dist=main_df.copy()

In [327]:
dengue_2025["Year_Month"]=dengue_2025["sample_date"].dt.to_period("M")

In [338]:
only_2025=dengue_2025[dengue_2025["sample_date"].dt.year>2024]

In [339]:
dengue_2025_dist=only_2025.groupby(by=["DISTRICT", "Year_Month"]).size().reset_index()

In [342]:
hist_dist=hist_dist.groupby(by=["New_District", "Year_Month"])["Dengue Cases"].sum().reset_index().rename(columns={
    "New_District":"DISTRICT",
    "Dengue Cases":"dengue_cases"
})

In [343]:
hist_dist["Year_Month"].min(), hist_dist["Year_Month"].max()

(Period('2019-01', 'M'), Period('2024-12', 'M'))

In [344]:
dengue_2025_dist["Year_Month"].min(), dengue_2025_dist["Year_Month"].max()


(Period('2025-01', 'M'), Period('2025-08', 'M'))

In [345]:
dengue_2025_dist["Year_Month"]=dengue_2025_dist["Year_Month"].astype(str)
hist_dist["Year_Month"]=hist_dist["Year_Month"].astype(str)

In [346]:
all_dist=pd.concat([hist_dist, dengue_2025_dist])

In [348]:
all_dist.duplicated(subset=["DISTRICT", "Year_Month"]).sum()

np.int64(0)

In [349]:
all_dist.to_csv("1_Data/VBD/clean/dengue_dist_summary.csv", index=False)


# Malaria 2024 & 2025 - 1st Jan 2024 - 30th Nov 2024
- Out of 34,441 villages in Rajasthan, only 740 villages (2.15%) reported any cases of Malaria.

- 101 sub-districts did not report any cases

- 4 districts - Jhalawar, Baran, Karauli, Dhaulpur did not report any cases in the 11 month period

In [447]:
# 2024 data until nov is already geodf
malaria_2024=gpd.read_file("1_Data/VBD/raw/malaria_patient_locations.gpkg")

In [448]:
malaria_2024.sample()

,sample_date,geometry
360,2024-11-28 00:00:00+00:00,POINT (73.63811 24.39117)


In [449]:
malaria_2024["sample_date"]=pd.to_datetime(malaria_2024["sample_date"]).dt.date

In [450]:
malaria_2024["sample_date"].min(), malaria_2024["sample_date"].max()

(datetime.date(2024, 1, 1), datetime.date(2024, 11, 30))

In [451]:
# mapping patient lat-long to blocks
malaria_2024=malaria_2024.sjoin(blocks_shp, how="left", predicate="within").drop(columns=["geometry", "index_right"])

In [452]:
# checking for nulls
malaria_2024.isna().sum()

,0
sample_date,0
NAME,0
DISTRICT,0
C_CODE11,0


In [453]:
# importing 2025 data
malaria_2025=pd.read_csv("1_Data/VBD/raw/malaria_2025.csv")

In [454]:
malaria_2025.columns

Index(['_index', 'district_name', 'block_name', 'phc_name', '_month',
       'case_count', 'address', 'screening_date', 'infectious_disease', 'lat',
       'lon', 'cases_last_7_days', 'district_original', 'block_original',
       'cases_next_15_days'],
      dtype='object')

In [455]:
malaria_2025=malaria_2025[["screening_date", 'lat',
       'lon']]

In [456]:
malaria_2025["Lat-Long"]=malaria_2025["lat"].astype(str)+","+malaria_2025["lon"].astype(str)

In [457]:
# Convert string lat-long entries to shapely Point objects
def latlong_to_point(value):
    if pd.isna(value):
        return None
    try:
        parts = value.split(",")
        if len(parts) != 2:
            return None  # malformed entry
        lat, lon = float(parts[0].strip()), float(parts[1].strip())
        return Point(lon, lat)  # Note: Point(x=lon, y=lat)
    except Exception:
        return None

# Apply conversion
malaria_2025["geometry"] = malaria_2025["Lat-Long"].apply(latlong_to_point)

In [458]:
malaria_2025=gpd.GeoDataFrame(malaria_2025, geometry="geometry", crs="EPSG:4326")

In [459]:
malaria_2025=malaria_2025[["screening_date", "geometry"]]

In [460]:
malaria_2025.rename(columns={"screening_date":"sample_date"} , inplace=True)

In [462]:
malaria_2025["sample_date"]=pd.to_datetime(malaria_2025["sample_date"]).dt.date

In [463]:
# map to blocks shp
malaria_2025=malaria_2025.sjoin(blocks_shp, how="left", predicate="within")

In [464]:
# checking for unmapped. blocks
malaria_2025.isna().sum()

,0
sample_date,0
geometry,0
index_right,0
NAME,0
DISTRICT,0
C_CODE11,0


In [465]:
# drop geometry, index cols
malaria_2025.drop(columns=['geometry', 'index_right'], inplace=True)

In [469]:
len(malaria_2024) + len(malaria_2025)

(2060, 834)

In [471]:
# combine 2024, 2025
all_malaria=pd.concat([malaria_2024, malaria_2025])

In [474]:
# export block, sample_date level
all_malaria.to_csv("1_Data/VBD/clean/malaria_cases_jan-nov_2024_2025.csv", index=False)

In [477]:
# agg to block, month
all_malaria["Year_Month"]=pd.to_datetime(all_malaria["sample_date"]).dt.to_period("M")

In [482]:
malaria_monthly=all_malaria.groupby(by=["DISTRICT", "NAME", "C_CODE11","Year_Month"]).size().reset_index().rename(columns={0:"malaria_cases"})

In [484]:
malaria_monthly.to_csv("1_Data/VBD/clean/malaria_monthwise_jan-nov_2024_2025.csv", index=False)

In [486]:
# agg to week
all_malaria["Year"] = pd.to_datetime(all_malaria["sample_date"]).dt.isocalendar().year
all_malaria["Week"] = pd.to_datetime(all_malaria["sample_date"]).dt.isocalendar().week
all_malaria["Year_Week"] = all_malaria["Year"].astype(str) + "-W" + all_malaria["Week"].astype(str).str.zfill(2)

In [488]:
malaria_weekly=all_malaria.groupby(by=["DISTRICT", "NAME", "C_CODE11", "Year", "Week", "Year_Week"]).size().reset_index().rename(columns={0:"malaria_cases"})

In [489]:
malaria_weekly.to_csv("1_Data/VBD/clean/malaria_weekwise_jan-nov_2024_2025.csv", index=False)

In [ ]:
# The End!